In [2]:
import os
import cv2
import cv2 as cv
import numpy as np
import argparse
import warnings
import time
from tqdm import tqdm

from src.anti_spoof_predict import AntiSpoofPredict
from src.generate_patches import CropImage
from src.utility import parse_model_name
warnings.filterwarnings('ignore')

In [10]:
model_test = AntiSpoofPredict(0)
pretr = 'resources/anti_spoof_models/4_0_0_80x80_MiniFASNetV1SE.pth'

In [6]:
sample = "../fas_face_dataset/fake/1.jpg"

In [11]:
dataset ="../fas_face_dataset/"

In [26]:
def pre_process(img_path):
    img = cv2.imread(path_img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (80,80))
    return img

In [49]:
def predict_fas(img):
    prediction = model_test.predict(img, model)
    label = np.argmax(prediction)
    confident_score = prediction[0][label]
    if label == 1:
        # REAL 
        return "real"
    else:
        # 2D / 3D fake
        return "fake"

In [12]:
from tqdm import tqdm

In [53]:
tp = 0
tn = 0
fn = 0
fp = 0

# dir = fake / real 

labels = os.listdir(dataset)
for label in labels:
    print(label)
    label_dir = os.path.join(dataset, label)
    images = os.listdir(label_dir)
    for image in tqdm(images):
        img_path = os.path.join(label_dir, image)
        img = pre_process(img_path)
        tic = time.time()
        prediction = predict_fas(img)
        # print(time.time() - tic)
        
        if prediction == "fake" and label == 'fake':
            tp += 1
        elif prediction == 'real' and label == 'fake':
            fn += 1
        elif prediction == 'fake' and label == 'real':
            fp += 1
        elif prediction == 'real' and label == 'real':
            tn += 1
            
print("tp:", tp)
print("fp:", fp)
print("fn:", fn)
print("tn:", tn)

fake


100%|██████████| 571/571 [00:58<00:00,  9.71it/s]


real


100%|██████████| 400/400 [00:41<00:00,  9.56it/s]

tp: 571
fp: 400
fn: 0
tn: 0


In [6]:
model_1 = 'resources/anti_spoof_models/2.7_80x80_MiniFASNetV2.pth'
model_2 = 'resources/anti_spoof_models/4_0_0_80x80_MiniFASNetV1SE.pth'
model = model_1


In [5]:
model_name = model.split('/')[-1]

h_input, w_input, model_type, scale = parse_model_name(model_name)


In [9]:
scale

2.7

In [7]:
def predict_fas_face(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image_bbox = model_test.get_bbox(img)
    
    model_name = model.split('/')[-1]
    h_input, w_input, model_type, scale = parse_model_name(model_name)
    param = {
        "org_img": img,
        "bbox": image_bbox,
        "scale": scale,
        "out_w": w_input,
        "out_h": h_input,
        "crop": True,
    }
    if scale is None:
        param["crop"] = False
    img = CropImage().crop(**param)
    
    prediction = model_test.predict(img, model)
    label = np.argmax(prediction)
    confident_score = prediction[0][label]
    if label == 1:
        # REAL 
        return "real"
    else:
        # 2D / 3D fake
        return "fake"

In [17]:
model = model_1

In [18]:
tp = 0
tn = 0
fn = 0
fp = 0

# dir = fake / real 

labels = os.listdir(dataset)
for label in labels:
    print(label)
    label_dir = os.path.join(dataset, label)
    images = os.listdir(label_dir)
    for image in tqdm(images):
        img_path = os.path.join(label_dir, image)
        tic = time.time()
        try:
            prediction = predict_fas_face(img_path)
        except:
            continue
        # print(time.time() - tic)
        
        if prediction == "fake" and label == 'fake':
            tp += 1
        elif prediction == 'real' and label == 'fake':
            fn += 1
        elif prediction == 'fake' and label == 'real':
            fp += 1
        elif prediction == 'real' and label == 'real':
            tn += 1
            
print("tp:", tp)
print("fp:", fp)
print("fn:", fn)
print("tn:", tn)

fake


100%|██████████| 600/600 [02:27<00:00,  4.07it/s]


real


100%|██████████| 438/438 [00:43<00:00, 10.01it/s]

tp: 439
fp: 4
fn: 161
tn: 434


In [13]:
print("tp:", tp)
print("fp:", fp)
print("fn:", fn)
print("tn:", tn)

tp: 439
fp: 4
fn: 161
tn: 434


In [ ]:
fake
100%|██████████| 571/571 [00:58<00:00,  9.70it/s]
real
100%|██████████| 400/400 [00:50<00:00,  7.92it/s]
tp: 333
fp: 187
fn: 238
tn: 213

In [ ]:
fake
100%|██████████| 571/571 [01:03<00:00,  8.97it/s]
real
100%|██████████| 400/400 [00:48<00:00,  8.25it/s]
tp: 429
fp: 289
fn: 142
tn: 111

In [2]:
def predict_two_model(img_path):
    model_test = AntiSpoofPredict(0)
    image_cropper = CropImage()
    image = cv2.imread(img_path)

    image_bbox = model_test.get_bbox(image)
    prediction = np.zeros((1, 3))
    # test_speed = 0
    # sum the prediction from single model's result
    for model in [model_1, model_2]:
        model_name = model.split("/")[-1]
        h_input, w_input, model_type, scale = parse_model_name(model_name)
        param = {
            "org_img": image,
            "bbox": image_bbox,
            "scale": scale,
            "out_w": w_input,
            "out_h": h_input,
            "crop": True,
        }
        if scale is None:
            param["crop"] = False
        img = image_cropper.crop(**param)

        
        prediction += model_test.predict(img, model)
        
    label = np.argmax(prediction)
    value = prediction[0][label]/2
    return "real" if label == 1 else "fake"


In [8]:
tp = 0
tn = 0
fn = 0
fp = 0

# dir = fake / real 

labels = os.listdir(dataset)
for label in labels:
    print(label)
    label_dir = os.path.join(dataset, label)
    images = os.listdir(label_dir)
    for image in tqdm(images):
        img_path = os.path.join(label_dir, image)
        try:
            prediction = predict_two_model(img_path)
        except: continue
        # print(time.time() - tic)
        
        if prediction == "fake" and label == 'fake':
            tp += 1
        elif prediction == 'real' and label == 'fake':
            fn += 1
        elif prediction == 'fake' and label == 'real':
            fp += 1
        elif prediction == 'real' and label == 'real':
            tn += 1
            
print("tp:", tp)
print("fp:", fp)
print("fn:", fn)
print("tn:", tn)

fake


  0%|          | 0/571 [00:00<?, ?it/s]

100%|██████████| 571/571 [01:48<00:00,  5.26it/s]


real


100%|██████████| 400/400 [01:26<00:00,  4.63it/s]

tp: 6
fp: 0
fn: 1
tn: 0


In [ ]:
fake
100%|██████████| 571/571 [01:54<00:00,  4.98it/s]
real
100%|██████████| 400/400 [01:23<00:00,  4.79it/s]
tp: 389
fp: 143
fn: 182
tn: 257